# BTC Price Predictor - Model Training Demo

This notebook demonstrates how to train the LSTM and Transformer models for Bitcoin price prediction.

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Add project root to path
sys.path.append('..')

# Import project modules
from src.data.collectors import BinanceDataCollector, CoinGeckoDataCollector
from src.data.features import FeatureEngineer
from src.models.lstm_model import LSTMModel
from src.models.transformer_model import TemporalFusionTransformer
from src.models.trainer import ModelTrainer

## 1. Data Collection

First, let's collect historical Bitcoin price data from Binance and CoinGecko.

In [ ]:
# Create data collectors
binance_collector = BinanceDataCollector()
coingecko_collector = CoinGeckoDataCollector()

# Fetch short-term data from Binance (hourly)
short_term_data = binance_collector.get_historical_klines(
    symbol="BTCUSDT",
    interval="1h",
    limit=1000  # Last 1000 hours
)

print(f"Fetched {len(short_term_data)} records from Binance")
short_term_data.head()

In [ ]:
# Fetch long-term data from CoinGecko (daily)
long_term_data = coingecko_collector.get_bitcoin_historical_data(
    days=365  # Last 365 days
)

print(f"Fetched {len(long_term_data)} records from CoinGecko")
long_term_data.head()

## 2. Feature Engineering

Now, let's create features for our models.

In [ ]:
# Create feature engineer
fe = FeatureEngineer()

# Process short-term data
short_term_features = fe.add_technical_indicators(short_term_data)
short_term_features = fe.create_time_features(short_term_features)
short_term_features = fe.create_target_variables(short_term_features, horizons=[1, 6, 12, 24])

print(f"Short-term features shape: {short_term_features.shape}")
print(f"Columns: {short_term_features.columns.tolist()[:10]}...")

In [ ]:
# Process long-term data
# For CoinGecko data, we need to adapt the feature engineering
long_term_features = long_term_data.copy()
long_term_features = fe.add_technical_indicators(long_term_features, price_col='price', volume_col='volume')
long_term_features = fe.create_target_variables(long_term_features, price_col='price', horizons=[1, 3, 7, 14])

print(f"Long-term features shape: {long_term_features.shape}")
print(f"Columns: {long_term_features.columns.tolist()[:10]}...")

## 3. Visualize the Data

Let's visualize the Bitcoin price and some technical indicators.

In [ ]:
# Plot Bitcoin price
plt.figure(figsize=(14, 7))
plt.plot(short_term_features['open_time'], short_term_features['close'])
plt.title('Bitcoin Price (Hourly)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.grid(True)
plt.show()

In [ ]:
# Plot technical indicators
plt.figure(figsize=(14, 10))

# Price and moving averages
plt.subplot(3, 1, 1)
plt.plot(short_term_features['open_time'], short_term_features['close'], label='Price')
plt.plot(short_term_features['open_time'], short_term_features['sma_7'], label='SMA 7')
plt.plot(short_term_features['open_time'], short_term_features['sma_30'], label='SMA 30')
plt.title('Price and Moving Averages')
plt.legend()
plt.grid(True)

# RSI
plt.subplot(3, 1, 2)
plt.plot(short_term_features['open_time'], short_term_features['rsi_14'])
plt.axhline(y=70, color='r', linestyle='-')
plt.axhline(y=30, color='g', linestyle='-')
plt.title('RSI (14)')
plt.grid(True)

# MACD
plt.subplot(3, 1, 3)
plt.plot(short_term_features['open_time'], short_term_features['macd'], label='MACD')
plt.plot(short_term_features['open_time'], short_term_features['macd_signal'], label='Signal')
plt.bar(short_term_features['open_time'], short_term_features['macd_hist'], alpha=0.3, label='Histogram')
plt.title('MACD')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 4. Prepare Data for Training

Let's prepare the data for training our models.

In [ ]:
# Create model trainer
trainer = ModelTrainer()

# Select features for LSTM model
lstm_feature_cols = [
    'open', 'high', 'low', 'close', 'volume',
    'sma_7', 'sma_14', 'ema_7', 'ema_14', 
    'rsi_14', 'macd', 'macd_signal',
    'hour_sin', 'hour_cos', 'day_sin', 'day_cos'
]

# Keep only columns that exist in the data
lstm_feature_cols = [col for col in lstm_feature_cols if col in short_term_features.columns]

# Target variable (24-hour price prediction)
target_col = 'future_price_24'

# Prepare data for LSTM model
X_train, y_train, X_val, y_val, X_test, y_test = trainer.prepare_data(
    df=short_term_features.dropna(),
    feature_cols=lstm_feature_cols,
    target_col=target_col,
    sequence_length=24,  # 24 hours of data
    test_size=0.2,
    validation_size=0.1
)

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, {y_val.shape}")
print(f"Test data shape: {X_test.shape}, {y_test.shape}")

## 5. Train LSTM Model

Now, let's train the LSTM model for short-term predictions.

In [ ]:
# Train LSTM model
lstm_model = trainer.train_lstm_model(
    X_train, y_train,
    X_val, y_val,
    model_name='lstm_24h',
    model_type='bidirectional',  # Use bidirectional LSTM
    lstm_units=128,
    dropout_rate=0.3,
    batch_size=32,
    epochs=50,
    patience=10
)

## 6. Evaluate LSTM Model

Let's evaluate the LSTM model on the test data.

In [ ]:
# Evaluate LSTM model
lstm_metrics = trainer.evaluate_model(lstm_model, X_test, y_test)
print("LSTM Model Metrics:")
for metric, value in lstm_metrics.items():
    print(f"  {metric}: {value}")

In [ ]:
# Make predictions on test data
y_pred = lstm_model.predict(X_test).flatten()

# Plot actual vs predicted prices
plt.figure(figsize=(14, 7))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title('LSTM Model: Actual vs Predicted Bitcoin Prices')
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

## 7. Train Transformer Model

Now, let's train the Transformer model for long-term predictions.

In [ ]:
# Select features for Transformer model
transformer_feature_cols = [
    'price', 'volume', 'market_cap',
    'sma_7', 'sma_30', 'ema_7', 'ema_30',
    'rsi_14', 'macd', 'macd_signal'
]

# Keep only columns that exist in the data
transformer_feature_cols = [col for col in transformer_feature_cols if col in long_term_features.columns]

# Target variable (7-day price prediction)
target_col = 'future_price_7'

# Prepare data for Transformer model
X_train_tf, y_train_tf, X_val_tf, y_val_tf, X_test_tf, y_test_tf = trainer.prepare_data(
    df=long_term_features.dropna(),
    feature_cols=transformer_feature_cols,
    target_col=target_col,
    sequence_length=30,  # 30 days of data
    test_size=0.2,
    validation_size=0.1
)

print(f"Training data shape: {X_train_tf.shape}, {y_train_tf.shape}")
print(f"Validation data shape: {X_val_tf.shape}, {y_val_tf.shape}")
print(f"Test data shape: {X_test_tf.shape}, {y_test_tf.shape}")

In [ ]:
# Train Transformer model
transformer_model = trainer.train_transformer_model(
    X_train_tf, y_train_tf,
    X_val_tf, y_val_tf,
    model_name='transformer_7d',
    n_heads=4,
    hidden_dim=128,
    dropout_rate=0.2,
    batch_size=16,
    epochs=50,
    patience=10
)

## 8. Evaluate Transformer Model

Let's evaluate the Transformer model on the test data.

In [ ]:
# Evaluate Transformer model
transformer_metrics = trainer.evaluate_model(transformer_model, X_test_tf, y_test_tf)
print("Transformer Model Metrics:")
for metric, value in transformer_metrics.items():
    print(f"  {metric}: {value}")

In [ ]:
# Make predictions on test data
y_pred_tf = transformer_model.predict(X_test_tf).flatten()

# Plot actual vs predicted prices
plt.figure(figsize=(14, 7))
plt.plot(y_test_tf, label='Actual')
plt.plot(y_pred_tf, label='Predicted')
plt.title('Transformer Model: Actual vs Predicted Bitcoin Prices')
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

## 9. Make a New Prediction

Let's use our trained models to make a new prediction.

In [ ]:
# Get the latest data
latest_data = binance_collector.get_historical_klines(
    symbol="BTCUSDT",
    interval="1h",
    limit=48  # Last 48 hours
)

# Process the data
latest_features = fe.add_technical_indicators(latest_data)
latest_features = fe.create_time_features(latest_features)

# Select features
latest_features_array = latest_features[lstm_feature_cols].values

# Create sequence for model input
sequence_length = 24
model_input = latest_features_array[-sequence_length:].reshape(1, sequence_length, len(lstm_feature_cols))

# Make prediction
predicted_price = lstm_model.predict(model_input)[0][0]

# Get current price
current_price = latest_data['close'].iloc[-1]

print(f"Current BTC Price: ${current_price:.2f}")
print(f"Predicted BTC Price (24h): ${predicted_price:.2f}")
print(f"Change: ${predicted_price - current_price:.2f} ({(predicted_price - current_price) / current_price * 100:.2f}%)")

## 10. Conclusion

In this notebook, we've demonstrated how to:

1. Collect Bitcoin price data from Binance and CoinGecko
2. Create features for our models
3. Train LSTM and Transformer models
4. Evaluate the models
5. Make new predictions

These models can be deployed in the BTC Price Predictor application to provide real-time predictions.